# Practice Exercise: Scikit-Learn 2
### Feature Selection and Ranking

### Objectives

As in the [SK2 Tutorial](https://www.featureranking.com/tutorials/machine-learning-tutorials/sk-part-2-feature-selection-and-ranking/), the goal of this practice notebook is to illustrate how you can perform feature selection (FS) and ranking using the relevant methods within `Scikit-Learn`. You will be using the cleaned "income data" from previous data preparation practices you will take a cross-validation (CV) approach. 

In the previous practices, you cleaned and transformed the raw `income data` and renamed the `income` column as `target` (with high income being the positive class). Including `target`, the cleaned data consists of 42 columns and 45,222 rows. Each column is numeric and between 0 and 1.

For FS methods other than the ones illustrated here, please refer to the official `Scikit-Learn` documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest).

### Machine Learning: The Art and The Science

If you notice things in our practice exercises that are different from those in the corresponding tutorials, it's OK. There are usually multiple ways of doing the right thing in ML, and more importantly, always keep in mind that machine learning is as much **art** as it is **science**!

### Instructions

- For all the questions below, you will use **stratified 5-fold cross-validation with no repetitions** and set the random state to 999 when applicable. 
- As the wrapper (that is, the intended classifier), you will use a decision tree with a `max_depth` of 5. Don't forget to set the `random_state` so that your results do not change from run to run.
- For ease of computation, you will first randomly sample 5,000 rows from this dataset. 
- You will then split this sample into two equal-sized datasets.
- You will use the first set for **training**: you will find the best 10-features using different methods using this train data. 
- You will use the second set for **testing**: you will perform cross-validation in a paired fashion using the test data and then you will compare the results using a paired t-test.
- For scoring, you will use AUC, that is, "area under the ROC curve". 

**Hint:** For a list of scorers as a **string** that you can pass into `cross_val_score()` or `GridSearchCV()` methods, please try this:
```Python
from sklearn import metrics 
metrics.SCORERS.keys()
```

### Some Bookkeeping

- Define a variable called `num_samples` and set it to 5000. You will use this variable when sampling a smaller subset of the full set of instances.
- Define a variable called `num_features` and set it to 10. You will perform all feature selection tasks by making use of this `num_features` variable.
- Define a variable called `scoring_metric` and set it to to `'roc_auc'`. You will set `scoring` option in all `cross_val_score()` functions to this `scoring_metric` variable.
- Define an object called `clf` and set its value to `DecisionTreeClassifier(max_depth=5, random_state=999)`. You will use this classifier as your wrapper when comparing performance of feature selection (FS) methods.

You can achieve these by running the code chunk below:
```Python
import numpy as np
num_samples = 5000
num_features = 10
scoring_metric = 'roc_auc'
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=5, random_state=999)
```

In [1]:
import numpy as np
num_samples = 5000
num_features = 10
scoring_metric = 'roc_auc'
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=5, random_state=999)

### Exercise 0: Modeling Preparation

- Read in the clean data `us_census_income_data_clean_encoded.csv` on GitHub [here](https://github.com/akmand/datasets). 
- Randomly sample the rows.
- Split the sampled data as 50% training set and the remaining 50% test set using a random seed of 999. 
- Remember to separate `target` during the splitting process. 

In [2]:
import pandas as pd
import io
import requests

# so that we can see all the columns
pd.set_option('display.max_columns', None) 

df_url = 'https://raw.githubusercontent.com/akmand/datasets/master/us_census_income_data_clean_encoded.csv'
url_content = requests.get(df_url, verify=False).content
df = pd.read_csv(io.StringIO(url_content.decode('utf-8')))

print(df.shape)
df.head().round(3)

c:\program files\python38\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


(45222, 42)


,age,education_num,race,gender,hours_per_week,native_country,capital,workclass_federal_gov,workclass_local_gov,workclass_private,workclass_self_emp_inc,workclass_self_emp_not_inc,workclass_state_gov,workclass_without_pay,marital_status_divorced,marital_status_married_af_spouse,marital_status_married_civ_spouse,marital_status_married_spouse_absent,marital_status_never_married,marital_status_separated,marital_status_widowed,occupation_adm_clerical,occupation_armed_forces,occupation_craft_repair,occupation_exec_managerial,occupation_farming_fishing,occupation_handlers_cleaners,occupation_machine_op_inspct,occupation_other_service,occupation_priv_house_serv,occupation_prof_specialty,occupation_protective_serv,occupation_sales,occupation_tech_support,occupation_transport_moving,relationship_husband,relationship_not_in_family,relationship_other_relative,relationship_own_child,relationship_unmarried,relationship_wife,target
0,0.301,0.800,1,1,0.398,1,0.063,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0.452,0.800,1,1,0.122,1,0.042,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0.288,0.533,1,1,0.398,1,0.042,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0.493,0.400,0,1,0.398,1,0.042,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0.151,0.800,0,0,0.398,0,0.042,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


In [3]:
df_sample = df.sample(n=num_samples, random_state=999).reset_index(drop=True)
df_sample.shape

(5000, 42)

In [4]:
from sklearn.model_selection import train_test_split
target = df_sample.target.values
Data_df = df_sample.drop(columns = 'target')
Data_numpy = Data_df.values
D_train, D_test, t_train, t_test = train_test_split(Data_numpy, 
                                                    target, 
                                                    test_size=0.5, 
                                                    random_state=999)

In [5]:
print(D_train.shape)
print(D_test.shape)
print(t_train.shape)
print(t_test.shape)

(2500, 41)
(2500, 41)
(2500,)
(2500,)


### Exercise 1

Assess the cross-validated performance of your DT classifier using the **test** data with all the features.

In [6]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn import feature_selection as fs

In [7]:
cv_method = StratifiedKFold(n_splits=5, shuffle=True, random_state=999)

In [8]:
cv_results_full = cross_val_score(estimator=clf,
                             X=D_test,
                             y=t_test, 
                             cv=cv_method, 
                             scoring=scoring_metric)

cv_results_full

array([0.81756035, 0.86543536, 0.88047532, 0.87261471, 0.91489938])

In [9]:
cv_perf_full = cv_results_full.mean().round(3)
cv_perf_full

0.87

### Exercise 2

- Select the top 10 features via the **F-Score** method using the **train** data.
- Evaluate the cross-validated performance of these features using your DT classifier on the **test** data.

**NOTE:** For this particular dataset, the F-Score will be "NaN" for one of the features due to some technical reasons (related to the nature of the F-distribution). For this reason, when you pass the `fs_fit_fscore.scores_` object in to the `np.argsort()` function, you will need to apply the `np.nan_to_num()` function first. This way, you will convert that "NaN" value to zero for a correct result. Specifically, you will need the following line:
```Python
fs_indices_fscore = np.argsort(np.nan_to_num(fs_fit_fscore.scores_))[::-1][0:num_features]

```

In [10]:
fs_fit_fscore = fs.SelectKBest(fs.f_classif, k=num_features)
fs_fit_fscore.fit_transform(D_train, t_train)
fs_indices_fscore = np.argsort(np.nan_to_num(fs_fit_fscore.scores_))[::-1][0:num_features]
# Let's see what these top features are
best_features_fscore = Data_df.columns[fs_indices_fscore].values
best_features_fscore

array(['marital_status_married_civ_spouse', 'relationship_husband',
       'education_num', 'marital_status_never_married',
       'relationship_own_child', 'age', 'hours_per_week', 'gender',
       'relationship_not_in_family', 'occupation_prof_specialty'],
      dtype=object)

In [11]:
cv_results_fscore = cross_val_score(estimator=clf,
                             X=D_test[:, fs_indices_fscore],
                             y=t_test, 
                             cv=cv_method, 
                             scoring=scoring_metric)
cv_perf_fscore = cv_results_fscore.mean().round(3)
cv_perf_fscore

0.855

### Exercise 3

- Select the top 10 features using the **Mutual Information** method using the **train** data.
- Evaluate the cross-validated performance of these features using your DT classifier on the **test** data.

In [12]:
fs_fit_mutual_info = fs.SelectKBest(fs.mutual_info_classif, k=num_features)
fs_fit_mutual_info.fit_transform(D_train, t_train)
fs_indices_mutual_info = np.argsort(fs_fit_mutual_info.scores_)[::-1][0:num_features]
# Let's see what these top features are
best_features_mutual_info = Data_df.columns[fs_indices_mutual_info].values
best_features_mutual_info

array(['marital_status_married_civ_spouse', 'capital',
       'relationship_husband', 'age', 'marital_status_never_married',
       'education_num', 'relationship_own_child', 'hours_per_week',
       'occupation_exec_managerial', 'relationship_not_in_family'],
      dtype=object)

In [13]:
cv_results_mutual_info = cross_val_score(estimator=clf,
                             X=D_test[:, fs_indices_mutual_info],
                             y=t_test, 
                             cv=cv_method, 
                             scoring=scoring_metric)
cv_perf_mutual_info = cv_results_mutual_info.mean().round(3)
cv_perf_mutual_info

0.872

### Exercise 4

- Select the top 10 features using the **Random Forest Importance** method (with `random_state=999`) with `n_estimators=100` using the **train** data.
- Evaluate the cross-validated performance of these features using your DT classifier on the **test** data.

In [14]:
from sklearn.ensemble import RandomForestClassifier
model_rfi = RandomForestClassifier(n_estimators=100, random_state=999)
model_rfi.fit(D_train, t_train)
fs_indices_rfi = np.argsort(model_rfi.feature_importances_)[::-1][0:num_features]
# Let's see what these top features are
best_features_rfi = Data_df.columns[fs_indices_rfi].values
best_features_rfi

array(['age', 'education_num', 'capital', 'hours_per_week',
       'marital_status_married_civ_spouse', 'relationship_husband',
       'marital_status_never_married', 'occupation_prof_specialty',
       'occupation_exec_managerial', 'workclass_private'], dtype=object)

In [15]:
cv_results_rfi = cross_val_score(estimator=clf,
                             X=D_test[:, fs_indices_rfi],
                             y=t_test, 
                             cv=cv_method, 
                             scoring=scoring_metric)
cv_perf_rfi = cv_results_rfi.mean().round(3)
cv_perf_rfi

0.871

### Exercise 5

Conduct 3 paired t-tests at a 95% significance level: the cross-validated performance with full set of features (using the **test** data) vs. each one of the three FS methods (evaluated again on the **test** data). 

But first remind yourself the performances of the FS methods by printing the 4 respective `cv_perf_?` variables.

Comment on performance of which FS method(s) is (are) statistically different from that of the full set of features. Does FS seem to result in any meaningful dimensionality reduction in this particular case?

**Hint:** Any p-value smaller than 0.05 indicates a statistically different result.

In [16]:
# make sure you are on Python 3.6+ for f-strings to work!
print(f'Full Set of Features (with {D_train.shape[1]} Features):', cv_perf_full)
print(f'Feature Selection with {num_features} Features:')
print('F-Score:', cv_perf_fscore)
print('Mutual Information:', cv_perf_mutual_info)
print('RFI:', cv_perf_rfi)

Full Set of Features (with 41 Features): 0.87
Feature Selection with 10 Features:
F-Score: 0.855
Mutual Information: 0.872
RFI: 0.871


In [17]:
from scipy import stats
print('P-Value for Full vs F-Score:', stats.ttest_rel(cv_results_full, cv_results_fscore).pvalue.round(3))
print('P-Value for Full vs Mutual Information:', stats.ttest_rel(cv_results_full, cv_results_mutual_info).pvalue.round(3))
print('P-Value for Full vs RFI:', stats.ttest_rel(cv_results_full, cv_results_rfi).pvalue.round(3))

P-Value for Full vs F-Score: 0.262
P-Value for Full vs Mutual Information: 0.21
P-Value for Full vs RFI: 0.372


### Exercise 6

Re-run your entire notebook with different combinations of different settings and see for yourself if FS is still meaningful. Some suggested changes are as follows:
- Change `num_samples` to 10000 or 20000.
- Change `num_features` to 5 or 20.
- Change `scoring_metric` to 'accuracy' or 'f1'.
- Change `max_depth` in DT to 3 or 10.
- Try different wrappers, such as KNN with different $k$ and $p$ values.

### Optional: Feature Selection in a Pipeline

Sometimes, we need to determine the number of features. In our example, should it be 10, 20 or 30? One approach is to write a for-loop which iterates a list of number of features. This is not efficient and prone to errors! Luckily we have `Pipeline`! **Note**: F-score seems to cause some issue in `Pipeline`. So, let's try another univariate feature selection method which is $\chi^{2}$ test: `fs.chiq`

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
pipe = Pipeline([
    ('fs', 'passthrough'),
    ('classifier', clf )
])
            
n_features = [10, 20, 30, 40]
param_grid = [
    {
        'fs': [fs.SelectKBest(fs.chi2)],
        'fs__k': n_features,
    }    
    
]
grid = GridSearchCV(pipe, 
                    n_jobs=1, 
                    param_grid=param_grid, 
                    cv=cv_method,
                    scoring=scoring_metric)
grid.fit(D_train, t_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=999, shuffle=True),
             estimator=Pipeline(steps=[('fs', 'passthrough'),
                                       ('classifier',
                                        DecisionTreeClassifier(max_depth=5,
                                                               random_state=999))]),
             n_jobs=1,
             param_grid=[{'fs': [SelectKBest(k=30,
                                             score_func=<function chi2 at 0x000001859F033EE0>)],
                          'fs__k': [10, 20, 30, 40]}],
             scoring='roc_auc')

We can obtain the best score and cv_results from the `grid`. Note that the mean scores from the grid correspond to `k=10, 20, 30, 40`.

In [19]:
mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print(grid.best_score_)

[0.83626222 0.85073711 0.871161   0.87107641]
0.8711610021220201


By callling `best_params_`, we can find that 30 features yield the best score.

In [20]:
grid.best_params_

{'fs': SelectKBest(k=30, score_func=<function chi2 at 0x000001859F033EE0>),
 'fs__k': 30}

We can calculate cross-validated score by setting estimator to the `grid`. Here is one convenience: we no longer need to slice the test data (`D_train`) as the `grid` contains the information of optimal features obtained from the pipeline!

In [21]:
cv_results_pip = cross_val_score(estimator=grid,
                                 X=D_test,
                                 y=t_test, 
                                 cv=cv_method, 
                                 scoring=scoring_metric)
cv_results_pip = cv_results_pip.mean().round(3)
cv_results_pip 

0.868

In Exercise 4, we developed a RandomForest to select the top 10 (`num_features`=10) features and subset the training set before fitting a Decision Tree model (`clf`). We can "chain" this process into a pipeline.

In [22]:
model = Pipeline([
  ('feature_selection', fs.SelectFromModel(RandomForestClassifier(n_estimators=100, 
                                                                  random_state=999,
                                                                  max_features=num_features))),
  ('classification', clf)
])
model.fit(D_train, t_train)

Pipeline(steps=[('feature_selection',
                 SelectFromModel(estimator=RandomForestClassifier(max_features=10,
                                                                  random_state=999))),
                ('classification',
                 DecisionTreeClassifier(max_depth=5, random_state=999))])

***
www.featureranking.com